<br>

# RAJs, Circunscrição Judiciária e Comarcas

Por meio do site [Quem Somos](https://www.tjsp.jus.br/QuemSomos/QuemSomos/RegioesAdministrativasJudiciarias) raspei as informações

<br>

Michel Metran\
Data: X.X.2020\
Atualizado em: 18.06.2024


In [ ]:
!pip3 install traquitanas

In [4]:
import pandas as pd
from selenium.webdriver.common.by import By

from sp_tjsp_divadmin.my_driver import Driver
from sp_tjsp_divadmin.my_functions import (
    find_text_between_parenthesis,
    keep_numbers,
)
from sp_tjsp_divadmin.my_paths import (
    adds_path,
    driver_path,
    logs_path,
    output_path_tab,
)

<br>

-----

## Drivers

Inicialmente crio um driver.

In [ ]:
driver = Driver(
    my_driver_path=driver_path,
    my_logs_path=logs_path,
    verify_ssl=False,
)
driver.add_extension_xpath(my_adds_path=adds_path)

In [6]:
# Go TJSP
URL = 'https://www.tjsp.jus.br/QuemSomos/QuemSomos/RegioesAdministrativasJudiciarias'
driver.get(URL)

<br>

----

## RAJs

Pego os dados das RAJs

In [7]:
def get_rajs_info(raj_xpath):
    """
    Retorna dicionário das Circuncrições Judiciárias

    :param raj_xpath: selenium.element
    :return: _description_
    """
    # Xpath
    raj_nome_xpath = raj_xpath.find_element(
        By.XPATH, './/p[@style="font-weight: bold"]'
    )
    diretor_xpath = raj_nome_xpath.find_element(
        By.XPATH, "./following-sibling::p"
    )
    email_xpath = diretor_xpath.find_element(By.XPATH, "./following-sibling::p")

    return {
        'raj_nome': raj_nome_xpath.text,
        'juiz_diretor_nome': diretor_xpath.text,
        'juiz_diretor_email': email_xpath.text,
    }

In [ ]:
# Localiza <div> irmã dos "Juízes Diretores"
juiz_xpath = driver.find_element(By.XPATH, '//*[text()="Juízes Diretores"]//..')
print(juiz_xpath.text)

As Comarcas e CJs estão agrupadas em `uls`.


In [ ]:
rajs_xpaths = juiz_xpath.find_element(
    By.XPATH, './following-sibling::div'
).find_elements(By.XPATH, './/div[contains(@style, "background")]')

print(f'São {len(rajs_xpaths)} RAJs')
rajs_xpaths

In [ ]:
df_raj = pd.DataFrame([get_rajs_info(raj_xpath=x) for x in rajs_xpaths])
df_raj

In [ ]:
# Adjust Data
df_raj['raj_nome'] = df_raj['raj_nome'].str.replace('–', '-')
df_raj['raj_nome'] = df_raj['raj_nome'].str.strip()
df_raj[['raj_sigla', 'raj_regiao']] = df_raj['raj_nome'].str.split(
    ' - ', expand=True
)

# Números
df_raj['id_raj'] = df_raj['raj_sigla'].apply(lambda x: keep_numbers(x))
df_raj['id_raj'] = df_raj['id_raj'].astype(int)

# Nome do Diretor
df_raj['juiz_diretor_nome'] = (
    df_raj['juiz_diretor_nome']
    .replace('Juiza diretora: ', '', regex=True)
    .replace('Juíza diretora: ', '', regex=True)
    .replace('Juiz diretor: ', '', regex=True)
)
df_raj['juiz_diretor_nome'] = df_raj['juiz_diretor_nome'].str.strip()

# Email do Diretor
df_raj['juiz_diretor_email'] = df_raj['juiz_diretor_email'].apply(
    lambda x: find_text_between_parenthesis(x)
)
df_raj['juiz_diretor_email'] = df_raj['juiz_diretor_email'].str.strip()

# Ordenação
df_raj = df_raj.sort_values(by='id_raj', ascending=True)
df_raj = df_raj.reset_index(drop=True)

# Results
df_raj.info()
df_raj.head()

In [ ]:
df_raj = df_raj[
    [
        # RAJ
        'id_raj',
        'raj_nome',
        'raj_sigla',
        'raj_regiao',
        'juiz_diretor_nome',
        'juiz_diretor_email',
    ]
]
df_raj

In [ ]:
df_raj = df_raj.drop_duplicates()
df_raj = df_raj.sort_values(by='id_raj')
df_raj = df_raj.reset_index(drop=True)
display(df_raj.head())

In [14]:
filename = 'RAJs'

df_raj.to_csv(
    output_path_tab / f'{filename}.csv',
    index=False,
)

df_raj.to_excel(
    output_path_tab / f'{filename}.xlsx',
    index=False,
    sheet_name=f'{filename}',
)

<br>

---

## Comarcas e CJs


In [15]:
list_dfs = []
for raj in rajs_xpaths:
    # Para cada RAJ, pega Comarcas e CJs
    comarcas_xpath = raj.find_element(
        By.XPATH, './following-sibling::ul'
    ).find_elements(By.XPATH, './li')

    # Cria tabela
    df_comarcas = pd.DataFrame(
        [x.text for x in comarcas_xpath], columns=['comarca_cirscunscricao']
    )
    df_comarcas['raj_nome'] = get_rajs_info(raj_xpath=raj)['raj_nome']
    list_dfs.append(df_comarcas)

In [ ]:
# Create Table
# df = pd.DataFrame(list_dfs)
df = pd.concat(list_dfs, ignore_index=True)

# Results
df.info()
df.head()

In [ ]:
# Adjust Data
df['raj_nome'] = df['raj_nome'].str.replace('–', '-')
df[['raj_sigla', 'raj_regiao']] = df['raj_nome'].str.split(' - ', expand=True)
df[['comarca', 'cj_sigla']] = df['comarca_cirscunscricao'].str.split(
    ' - ', expand=True
)

# Números
df['id_raj'] = df['raj_sigla'].apply(lambda x: keep_numbers(x))
df['id_raj'] = df['id_raj'].astype(int)

df['id_cj'] = df['cj_sigla'].apply(lambda x: keep_numbers(x))
df.loc[df['id_cj'] == '', 'id_cj'] = '0'
df['id_cj'] = df['id_cj'].astype(int)

# Results
df.info()
df.head()

In [ ]:
df['cj_nome'] = df['cj_sigla'].replace(
    'CJ', 'Circunscrição Judiciária', regex=True
)

# Results
df.info()
df.head()

In [ ]:
df = df[
    [
        # RAJ
        'id_raj',
        'raj_nome',
        'raj_sigla',
        'raj_regiao',
        #'juiz_diretor',
        #'email',
        # CJ
        'id_cj',
        'cj_nome',
        'cj_sigla',
        # Comarca
        'comarca',
    ]
]

# Results
df.info()
df.head()

<br>


----

## CJs


In [ ]:
df_cjs = df[
    [
        # CJ
        'id_cj',
        'cj_nome',
        'cj_sigla',
        'id_raj',
    ]
]

# df_cjs.loc[df_cjs['cj_id'] == '', 'cj_id'] = '0'
# df_cjs['cj_id'] = df_cjs['cj_id'].copy()
# df_cjs['cj_id'] = df_cjs['cj_id'].astype('int')
df_cjs = df_cjs.drop_duplicates()
df_cjs = df_cjs.sort_values(by='id_cj')
df_cjs = df_cjs.reset_index(drop=True)

# Results
df_cjs.head()
df_cjs.info()

In [21]:
filename = 'Circunscrições Judiciárias'

df_cjs.to_csv(
    output_path_tab / f'{filename}.csv',
    index=False,
)

df_cjs.to_excel(
    output_path_tab / f'{filename}.xlsx',
    sheet_name=f'{filename}',
    index=False,
)

<br>

----

## Comarcas


In [ ]:
df_comarca = df[
    [
        # Comarca
        'comarca',
        'id_cj',
    ]
]

# Deleta Duplicatas
df_comarca = df_comarca.drop_duplicates()

# Renomear
df_comarca = df_comarca.rename(
    {
        'comarca': 'comarca_tjsp',
    },
    axis='columns',
)

# Ordena
df_comarca = df_comarca.iloc[
    df_comarca['comarca_tjsp'].str.normalize('NFKD').argsort()
]
df_comarca = df_comarca.reset_index(drop=True)

# Results
df_comarca.head()
df_comarca.info()

In [23]:
filename = 'Comarcas e CJs'

df_comarca.to_csv(
    output_path_tab / f'{filename}.csv',
    index=False,
)

df_comarca.to_excel(
    output_path_tab / f'{filename}.xlsx',
    index=False,
    sheet_name=f'{filename}',
)

In [ ]:
comarcas = df['comarca']

print(f'São {len(set(comarcas))} comarcas')

In [25]:
driver.quit()